In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
#Machine Learning Models
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier, plot_importance
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import time
from collections import Counter
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

In [32]:
#URL
df1 = pd.read_csv('url_features.csv').sort_values(by='url')
# df1 = pd.get_dummies(df1, columns=['tld_type'], prefix='tld')
# X1 = df1.drop(['url','label'], axis=1)
# y1 = df1['label']

In [33]:
#HTML
df2 = pd.read_csv('new_html_features.csv').sort_values(by='url')
# X2 = df2.drop(['label', 'url'], axis=1)
# y2 = df2['label']

In [34]:
df3 = pd.merge(df1, df2, on="url")

In [35]:
df3 = pd.get_dummies(df3, columns=['tld_type'], prefix='tld')
df3 = df3.rename(columns={'label_x': 'label'})

In [36]:
# df3 = pd.get_dummies(df3, columns=['tld_type'], prefix='tld')
# X3 = df3.drop(['url','label','label_y'], axis=1)
# y3 = df3['label']

In [37]:
# 25% - 174,268
n_legit = 168040
n_phish = 5228

In [38]:
legit_df3 = df3[df3['label'] == 0]
phish_df3 = df3[df3['label'] == 1]

sampled_legit3 = legit_df3.sample(n=n_legit, random_state=42)
sampled_phish3 = phish_df3.sample(n=n_phish, random_state=42)

In [39]:
reduced_df3 = pd.concat([sampled_legit3, sampled_phish3]).sample(frac=1, random_state=42).reset_index(drop=True)

X3 = reduced_df3.drop(['url','label', 'label_y'], axis=1)
y3 = reduced_df3['label']

In [40]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, stratify=y3, test_size=0.2, random_state=42)

In [57]:
# smote = SMOTE(sampling_strategy='auto', random_state=42)  
# X_resampled3, y_resampled3 = smote.fit_resample(X_train3, y_train3)

# smote_tomek = SMOTETomek(random_state=42)
# X_resampled3, y_resampled3 = smote_tomek.fit_resample(X_train3, y_train3)

# smote_enn = SMOTEENN(random_state=42)
# X_resampled3, y_resampled3 = smote_enn.fit_resample(X_train3, y_train3)



# adasyn = ADASYN(random_state=42, n_neighbors=5)
# X_resampled3, y_resampled3 = adasyn.fit_resample(X_train3, y_train3)


# rus = RandomUnderSampler(random_state=42)
# X_resampled3, y_resampled3 = rus.fit_resample(X_train3, y_train3)

ros = RandomOverSampler(random_state=42)
X_resampled3, y_resampled3 = ros.fit_resample(X_train3, y_train3)



# ros = RandomOverSampler(sampling_strategy={1: 5000}, random_state=42)
# rus = RandomUnderSampler(sampling_strategy={0: 5000, 1: 5000}, random_state=42)
# pipeline = Pipeline(steps=[('ros', ros), ('rus', rus)])
# X_resampled3, y_resampled3 = pipeline.fit_resample(X_train3, y_train3)


In [58]:
models = {
    "XGBoost": XGBClassifier(n_estimators= 281, max_depth= 14, learning_rate= 0.0888992418241481, subsample= 0.9134952932290352, colsample_bytree= 0.6533354924947662, gamma= 0.25380562179671745, random_state=42),
    "LightGBM": LGBMClassifier(num_leaves= 96, max_depth= 15, learning_rate= 0.16099354321038145, n_estimators= 292, subsample= 0.995245362847253, colsample_bytree= 0.883261087073046, min_child_samples= 25,random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators= 206, max_depth= 95, min_samples_split= 3, min_samples_leaf= 1, max_features= 'sqrt', random_state=42),
    # "K-Nearest Neighbor": KNeighborsClassifier(n_neighbors= 18, weights= 'distance', p= 1),
    "Decision Tree": DecisionTreeClassifier(max_depth= 20, min_samples_split= 11, min_samples_leaf= 2, criterion='entropy', random_state=42)
}

In [59]:
# print("URL & HTML")
# for name, model in models.items():
#     print(f"\n{name} ")
#     model.fit(X_train3, y_train3)
#     pred = model.predict(X_test3)
#     print(classification_report(y_test3, pred))


print("URL and HTML")
for name, model in models.items():
    print(f"\n{name} ")
    model.fit(X_resampled3, y_resampled3)
    # model.fit(X_train3, y_train3)
    pred = model.predict(X_test3)
    print(classification_report(y_test3, pred))

URL and HTML

XGBoost 
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     33608
           1       0.55      0.84      0.67      1046

    accuracy                           0.97     34654
   macro avg       0.77      0.91      0.83     34654
weighted avg       0.98      0.97      0.98     34654


LightGBM 
[LightGBM] [Info] Number of positive: 134432, number of negative: 134432
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035721 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6034
[LightGBM] [Info] Number of data points in the train set: 268864, number of used features: 187
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain